### Notebook to exemplify the Server exchange latency calculation. 
This notebook outlines latencies not specific to our specific algorithm or autospreader. They are inherent to each trade<br>
The definitions of the exchanges calculated here are inserted below
-   Server to exchange latency <br> 
    The time that an order takes to go from our trading machine to the exchange. It is worthwhile noting that this latency includes risk checks by the exchange and TT server. It can be computed in this  by passing a csv to the process\_csv function and then calling server\_to\_exchange\_latency function using the following: server_to_exchange_latency(orders_df,"server_to_exch_latency",["NEW"])<br>The parameters are: <br>
    - orders_df: The dataframe of the AuditTrail.
    - "server_to_exch_latency": keyword specifying latency computed
    - ['NEW']: Specifies that we are considering server to exchange latency of new orders, not replaced. 
-   Exchange Latency <br>
    The time taken by the exchange to process an order plus the time it takes for the order to reach our order connector. There are no risk checks present in this latency. It can be computed in this notebook by passing a csv to the process\_csv function and then calling exchange\_latency function using the following: exchange_latency(orders_df, ["NEW"],"exch_latency")<br> The parameters are: <br>
    - orders_df: The dataframe of the AuditTrail.
    - ["NEW]: Specifies that we are considering exchange latency of new orders, not replaced. 
    - "exch_latency": keyword specifying latency computed <br>
    <br>

It is important to note that the timestamps of the exchange are recorded in **milliseconds** and the timestamps on our TT server are recorded in **nanoseconds**. This can lead to a negative server\_exchange latency being observed. Consider for example: <br> 
The time the order leaves the exchange is **13:00:00.0007**. i.e 700 nanoseconds past 1pm. This will be **recorded** as **13:00:00.001**. Which is one millisecond past 1pm. <br>
Now lets say it takes the order **200** ns to reach our TT server once it is recorded leaving the exchange. Now we have recorded a time of confirmation of **13:00:00.0009**. This will lead to a **negative** offset of **100 ns** being calculated (**13:00:00.0009**-**13:00:00.001**). Which is clearly not possible but occurs due to the rounding error of the exchange.

In [1]:
# imports
import numpy as np
import pandas as pd
from latency_config import (
    ATColumns,
    ETypes,
    SynchMode,
    QuotesColumns,
    TradesColumns,
    QUOTE_ORDERS,
    ORDER_TYPE_TO_SIDE,
    AT_TABLE_PKEYS,
)
from analysis_metrics.base.base_latency import BaseLatency
import datetime as dt
from datetime import timezone
from audit_trail_manipulation import ATManipulation
import utils

In [2]:
# Function to calculate server to exchange latency. 
def server_to_exchange_latency(
        df: pd.DataFrame, latency_type: str, valid_orders: list = ["NEW"]
    ):
        df = BaseLatency.get_not_ase_rows(df)
        server_to_exchange_latencies = df[
            df[ATColumns.EXEC_TYPE].isin(valid_orders)
        ].copy()
        # First filter to discard corrupt data
        server_to_exchange_latencies = server_to_exchange_latencies[
            server_to_exchange_latencies[ATColumns.EXCHANGE_TIME]
            >= server_to_exchange_latencies[ATColumns.ORIGINAL_TIME]
        ]
        server_to_exchange_latencies[latency_type] = BaseLatency.to_millisec(
            server_to_exchange_latencies[ATColumns.EXCHANGE_TIME]
            - server_to_exchange_latencies[ATColumns.ORIGINAL_TIME]
        )
        server_to_exchange_latencies = BaseLatency._reorder_cols(
            server_to_exchange_latencies, latency_type=latency_type
        )
        return server_to_exchange_latencies

In [3]:
def exchange_latency(
        df: pd.DataFrame, exch_latency_valid_orders: list, latency_type: str
    ):
        # outrights dataframe
        # New, Cancel y Replace de outrights : exchlatency +/- 1ms
        df = BaseLatency.get_not_ase_rows(df)
        exch_latencies = df[
            df[ATColumns.EXEC_TYPE].isin(exch_latency_valid_orders)
        ].copy()
        # NOTE: Exchange time in ms. Our timestamps are measured in nanoseconds. Can cause negative offset, 
        exch_latencies[latency_type] = BaseLatency.to_millisec(
            exch_latencies[ATColumns.TIME_SENT]
            - exch_latencies[ATColumns.EXCHANGE_TIME]
        )
        exch_latencies = BaseLatency._reorder_cols(exch_latencies, latency_type=latency_type)
        return exch_latencies

In [5]:
# Read Comma separated file. 
def process_csv(csv_name):
    orders_df = pd.read_csv(csv_name, index_col="dtime")
    orders_df[ATColumns.TIME_SENT]=orders_df.index
    orders_df = BaseLatency.prepare_loaded(orders_df)
    return orders_df
orders_df=process_csv("raw_df.csv")

In [6]:
# To run server to exchange latency analysis. 
server_exch_latencies=server_to_exchange_latency(orders_df,"server_to_exch_latency",["NEW"])
print(server_exch_latencies)

                                  server_to_exch_latency   account instrument  \
dtime                                                                           
2024-02-02 13:30:01.079000+00:00                     0.0  CLX44550      FFX24   
2024-02-02 13:30:01.091000+00:00                     0.0  CLX44550     SFRU24   
2024-02-02 13:30:01.092000+00:00                     0.0  CLX44550     SFRU24   
2024-02-02 13:30:01.092000+00:00                     0.0  CLX44550     SFRU24   
2024-02-02 13:30:01.092000+00:00                     0.0  CLX44550     SFRU24   
2024-02-02 13:30:01.092000+00:00                     0.0  CLX44550     SFRU24   

                                 product  order_price  order_price_ticks  \
dtime                                                                      
2024-02-02 13:30:01.079000+00:00      FF       95.835            19167.0   
2024-02-02 13:30:01.091000+00:00     SFR       95.730            19146.0   
2024-02-02 13:30:01.092000+00:00     SFR       

In [7]:
# To run exchange latency analysis
exch_latencies=exchange_latency(orders_df, ["NEW"],"exch_latency")
print(exch_latencies['exch_latency'])


dtime
2024-02-02 13:30:01.079000+00:00     8.0
2024-02-02 13:30:01.091000+00:00    12.0
2024-02-02 13:30:01.092000+00:00    13.0
2024-02-02 13:30:01.092000+00:00    13.0
2024-02-02 13:30:01.092000+00:00    13.0
2024-02-02 13:30:01.092000+00:00    13.0
Name: exch_latency, dtype: float64
